# 🤖 AutoGen Basic Concepts Tutorial

## Welcome to AutoGen!

This notebook introduces the fundamental concepts of Microsoft AutoGen - a framework for creating multi-agent conversational systems.

### What You'll Learn:
1. **Core AutoGen Concepts**: Agents, conversations, and configurations
2. **Basic Agent Types**: UserProxyAgent and AssistantAgent
3. **Simple Conversations**: Two-agent interactions
4. **Code Execution**: How agents can run code
5. **Group Chats**: Multiple agents working together

---

## 🔧 Setup and Installation

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Import required libraries
import os
import autogen
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ AutoGen and dependencies imported successfully!")
print(f"📦 AutoGen version: {autogen.__version__}")

## ⚙️ Configuration Setup

### Key Concept: LLM Configuration

AutoGen agents need to know how to connect to language models. We configure this once and reuse it for all agents.

In [ ]:
# Configuration for connecting to OpenAI
config_list = [
    {
        "model": "gpt-3.5-turbo",  # You can also use "gpt-4" if you have access
        "api_key": os.environ.get("OPENAI_API_KEY"),
    }
]

# LLM configuration that will be used by our agents
llm_config = {
    "config_list": config_list,
    "temperature": 0.7,  # Controls randomness (0.0 = deterministic, 1.0 = very random)
    "timeout": 120,      # Maximum time to wait for a response
}

print("🔑 LLM configuration ready!")
print(f"🎯 Using model: {config_list[0]['model']}")
print(f"🌡️ Temperature: {llm_config['temperature']}")

## 🤝 Basic Concept 1: Two-Agent Conversation

### UserProxyAgent vs AssistantAgent

- **UserProxyAgent**: Represents the human user, can execute code
- **AssistantAgent**: AI assistant that generates responses and solutions

In [ ]:
# Create a UserProxyAgent (represents you, the user)
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message="A human user who provides tasks and feedback.",
    code_execution_config={
        "last_n_messages": 2,     # Only look at last 2 messages for code
        "work_dir": "coding",     # Directory to run code in
        "use_docker": False       # Set to True for better security
    },
    human_input_mode="NEVER",     # NEVER, ALWAYS, or TERMINATE
)

# Create an AssistantAgent (AI assistant)
assistant = autogen.AssistantAgent(
    name="Assistant",
    system_message="""You are a helpful AI assistant. 
    You can help with coding, analysis, and problem-solving.
    When you need to run code, write Python code and the user will execute it for you.""",
    llm_config=llm_config,
)

print("👥 Basic agents created!")
print(f"🧑 User Proxy: {user_proxy.name}")
print(f"🤖 Assistant: {assistant.name}")

### Let's Have Our First Conversation!

In [ ]:
# Start a simple conversation
print("🗣️ Starting a basic two-agent conversation...\n")

# The user proxy initiates the conversation
chat_result = user_proxy.initiate_chat(
    assistant,
    message="""Hello! Can you help me calculate the fibonacci sequence up to the 10th number? 
    Please write Python code to do this and explain how it works."""
)

print("\n✅ Conversation completed!")

## 🔄 Basic Concept 2: Code Execution

### How AutoGen Handles Code

1. Assistant writes code in markdown code blocks
2. UserProxy detects the code and executes it
3. Results are shared back with the assistant
4. Conversation continues based on results

In [ ]:
# Let's try a more complex coding task
print("🔢 Testing code execution capabilities...\n")

coding_task = """
I need help with data analysis. Can you:
1. Create a small dataset of student grades
2. Calculate the average, median, and standard deviation
3. Create a simple visualization

Please write and run the Python code to accomplish this.
"""

chat_result = user_proxy.initiate_chat(
    assistant,
    message=coding_task
)

print("\n📊 Data analysis task completed!")

## 👥 Basic Concept 3: Group Chat (Multiple Agents)

### Beyond Two Agents

Group chats allow multiple agents to collaborate on complex tasks. Each agent can have specialized roles.

In [ ]:
# Create specialized agents for a group chat

# Teacher agent - explains concepts
teacher = autogen.AssistantAgent(
    name="Teacher",
    system_message="""You are a patient teacher who explains programming concepts clearly.
    Focus on education and helping students understand the 'why' behind solutions.""",
    llm_config=llm_config,
)

# Coder agent - writes code
coder = autogen.AssistantAgent(
    name="Coder",
    system_message="""You are an expert programmer who writes clean, efficient code.
    Focus on practical implementation and best practices.""",
    llm_config=llm_config,
)

# Reviewer agent - checks and improves code
reviewer = autogen.AssistantAgent(
    name="Reviewer",
    system_message="""You are a code reviewer who focuses on quality, security, and optimization.
    Point out potential issues and suggest improvements.""",
    llm_config=llm_config,
)

print("👨‍🏫 Specialized agents created:")
print(f"📚 Teacher: {teacher.name}")
print(f"💻 Coder: {coder.name}")
print(f"🔍 Reviewer: {reviewer.name}")

In [ ]:
# Set up the group chat
groupchat = autogen.GroupChat(
    agents=[user_proxy, teacher, coder, reviewer],
    messages=[],
    max_round=12,  # Maximum number of back-and-forth exchanges
    speaker_selection_method="round_robin",  # or "auto" for automatic selection
)

# Create a manager to coordinate the group chat
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
    system_message="You coordinate the conversation between team members."
)

print("🎭 Group chat set up with 4 participants!")
print(f"🔄 Max rounds: {groupchat.max_round}")
print(f"🎯 Speaker selection: {groupchat.speaker_selection_method}")

In [ ]:
# Start a group discussion
print("🎪 Starting group chat discussion...\n")

group_task = """
Our team needs to create a simple calculator program that can:
1. Add, subtract, multiply, and divide two numbers
2. Handle division by zero errors
3. Have a user-friendly interface

Teacher: Please explain the concept first
Coder: Then implement the solution
Reviewer: Finally review and suggest improvements

Let's work together on this!
"""

chat_result = user_proxy.initiate_chat(
    manager,
    message=group_task
)

print("\n🎉 Group collaboration completed!")

## 🎛️ Basic Concept 4: Configuration Options

### Understanding Key Parameters

Let's explore the important configuration options that control agent behavior.

In [ ]:
# Demonstrate different human input modes
print("🎮 Different Human Input Modes:\n")

# Mode 1: NEVER (fully automated)
auto_agent = autogen.UserProxyAgent(
    name="AutoAgent",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "auto_coding"}
)

# Mode 2: ALWAYS (requires human input for every response)
interactive_agent = autogen.UserProxyAgent(
    name="InteractiveAgent", 
    human_input_mode="ALWAYS",
    code_execution_config={"work_dir": "interactive_coding"}
)

# Mode 3: TERMINATE (human can choose when to end)
selective_agent = autogen.UserProxyAgent(
    name="SelectiveAgent",
    human_input_mode="TERMINATE",
    code_execution_config={"work_dir": "selective_coding"}
)

print("🤖 NEVER mode: Fully automated, no human intervention")
print("👤 ALWAYS mode: Human input required for every step")
print("🛑 TERMINATE mode: Human can choose when to stop")

### Temperature and Creativity Settings

In [ ]:
# Create agents with different creativity levels

# Conservative agent (low temperature)
conservative_config = {
    "config_list": config_list,
    "temperature": 0.1,  # Very deterministic
}

conservative_agent = autogen.AssistantAgent(
    name="ConservativeAgent",
    system_message="You give precise, consistent answers with minimal creativity.",
    llm_config=conservative_config,
)

# Creative agent (high temperature)
creative_config = {
    "config_list": config_list,
    "temperature": 0.9,  # Very creative
}

creative_agent = autogen.AssistantAgent(
    name="CreativeAgent",
    system_message="You give creative, varied responses with lots of ideas.",
    llm_config=creative_config,
)

print("🎯 Agent personalities:")
print(f"🔒 Conservative (temp=0.1): Consistent, predictable responses")
print(f"🎨 Creative (temp=0.9): Varied, imaginative responses")

## 🔍 Basic Concept 5: Conversation Analysis

### Understanding What Happened

After a conversation, we can analyze the results to understand the interaction.

In [ ]:
# Function to analyze conversation results
def analyze_conversation(chat_result):
    """Simple analysis of a completed conversation"""
    
    print("📋 Conversation Analysis:")
    print("=" * 40)
    
    # Check if we have chat history
    if hasattr(chat_result, 'chat_history'):
        messages = chat_result.chat_history
        print(f"📨 Total messages: {len(messages)}")
        
        # Count messages by speaker
        speakers = {}
        for msg in messages:
            speaker = msg.get('name', 'Unknown')
            speakers[speaker] = speakers.get(speaker, 0) + 1
        
        print("\n👥 Messages by speaker:")
        for speaker, count in speakers.items():
            print(f"  {speaker}: {count} messages")
        
        # Show the conversation flow
        print("\n🔄 Conversation flow:")
        for i, msg in enumerate(messages[-5:], 1):  # Show last 5 messages
            speaker = msg.get('name', 'Unknown')
            content_preview = msg.get('content', '')[:50] + "..."
            print(f"  {i}. {speaker}: {content_preview}")
    
    else:
        print("No detailed chat history available")
    
    print("\n✅ Analysis complete!")

# Let's analyze our last conversation
if 'chat_result' in locals():
    analyze_conversation(chat_result)
else:
    print("💡 Run a conversation first, then we can analyze it!")

## 🎯 Practice Exercise: Build Your Own Simple Chat

Now it's your turn! Try creating your own agents and conversation.

In [ ]:
# Your turn! Create a simple math tutor system

# TODO: Create a MathTutor agent
math_tutor = autogen.AssistantAgent(
    name="MathTutor",
    system_message="""You are a friendly math tutor. 
    Help students solve math problems step by step.
    Always explain your reasoning and encourage learning.""",
    llm_config=llm_config,
)

# TODO: Create a Student agent (UserProxy)
student = autogen.UserProxyAgent(
    name="Student",
    system_message="A student learning mathematics.",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "math_practice"}
)

print("🎓 Math tutoring system ready!")
print("Try asking the tutor to help with a math problem.")

In [ ]:
# Practice conversation
math_problem = """
Hi! I need help solving this quadratic equation: x² + 5x + 6 = 0

Can you:
1. Show me how to solve it step by step
2. Verify the answer by substituting back
3. Explain what the solutions mean graphically
"""

practice_result = student.initiate_chat(
    math_tutor,
    message=math_problem
)

print("\n🎉 Math tutoring session completed!")

## 📚 Summary: AutoGen Basic Concepts

### What We Learned:

1. **🤖 Agent Types**:
   - `UserProxyAgent`: Represents users, can execute code
   - `AssistantAgent`: AI assistants with specialized roles

2. **⚙️ Key Configurations**:
   - `llm_config`: How agents connect to language models
   - `temperature`: Controls creativity/randomness
   - `human_input_mode`: NEVER, ALWAYS, or TERMINATE
   - `code_execution_config`: How and where to run code

3. **💬 Conversation Types**:
   - Two-agent chats: `agent1.initiate_chat(agent2)`
   - Group chats: Multiple agents with a manager

4. **🔧 Code Execution**:
   - Agents can write and execute Python code
   - Results are shared automatically
   - Useful for data analysis, calculations, and testing

### Next Steps:

- 🚀 Try the advanced demo notebook for complex scenarios
- 🔨 Experiment with different agent personalities
- 🌐 Explore external tool integrations
- 📊 Build domain-specific agent teams

---

**Happy coding with AutoGen! 🎉**

## 🛠️ Troubleshooting Common Issues

### Issue 1: "No API key found"
```python
# Make sure your .env file contains:
# OPENAI_API_KEY=your_actual_api_key_here
```

### Issue 2: "Code execution failed"
```python
# Check that your work directory exists and is writable
# Consider setting use_docker=True for better isolation
```

### Issue 3: "Conversation doesn't end"
```python
# Adjust max_round in GroupChat
# Use human_input_mode="TERMINATE" to control ending
```

### Issue 4: "Agents aren't collaborating well"
```python
# Improve system messages to clarify roles
# Use custom speaker selection for better flow
```